In [7]:
include("src/CombinedSubexpressions.jl")

CombinedSubexpressions

In [8]:
@which broadcast(+, [1,2,3], [4,5,6])

broadcast(f, As...) at broadcast.jl:230

In [2]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[2], line 19:
    function foo(x) # In[2], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [3]:
CombinedSubexpressions.cacheify(expr)

quote 
    begin  # In[2], line 19:
        function foo(x)
            ##gx#270 = g(x)
            ##f##gx#270#271 = f(##gx#270)
            ##h##f##gx#270#271##gx#270#272 = h(##f##gx#270#271,##gx#270)
            begin  # In[2], line 20:
                [h(##h##f##gx#270#271##gx#270#272,i) == i for i = 1:5]
            end
        end
    end
end

In [4]:
@CombinedSubexpressions.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#273 = f(x)
        begin  # In[4], line 2:
            [##fx#273 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [5]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [6]:
@CombinedSubexpressions.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#274 = g(x)
    ##f##gx#274#275 = f(##gx#274)
    ##h##f##gx#274#275##gx#274#276 = h(##f##gx#274#275,##gx#274)
    begin  # In[6], line 2:
        [h(##h##f##gx#274#275##gx#274#276,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false